# Bootstrap, L=5

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import Operator

from kraus_channels import KrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver

from quantum_tools import  resample
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, CorruptionMatrix
from quantum_circuits import integrable_circuit


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [3]:
def load_data(filename, n, seed, L):
    with open(filename, 'rb') as f:
        data = pkl.load(f)


    data = marginalize_counts(data, 0)

    targets = counts_to_probs(data)
    targets_spam = targets[:6**n]
    targets_map = targets[6**n:]

    np.random.seed(seed)
    
    circuit_target = integrable_circuit(n+1, L)
    unitary = Operator(circuit_target).data

    inputs_spam, _ = generate_pauliInput_circuits(n)
        
    inputs_map, circuit_list_map = (
                generate_pauli_circuits(n, None, N=5000-6**n)
        )
    
    return inputs_spam, targets_spam, inputs_map, targets_map, unitary

def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue()], sample_freq=100),
                )

    model.train(inputs = inputs,
                targets = targets,
                inputs_val = [inputs],
                targets_val = [targets],
                num_iter = num_iter,
                N = 500,
                verbose=verbose
                )
    
    return model

In [7]:
path = '../data/chaos_exp_reorder/5q/integrable_L=5/'
n = 4
d = 2**n
L = 5
bootstrap_samples = 10

spam_list = []
model_list = []

i = 0 # first map

seed = 42 + i

inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

tf.random.set_seed(seed)
for j in range(bootstrap_samples):
    targets_spam_bs = resample(targets_spam, 12000)
    targets_map_bs = resample(targets_map, 12000)

    spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
    spam_list.append(spam_model) 


    model = fit_model(inputs_map, 
                    targets_map_bs, 
                    spam_model,
                    num_iter = 3000, 
                    verbose=False)
    model_list.append(model)


model_saver(spam_list, f'models/integrable_spam_L=5_bootstrap_seed{seed}.model')
model_saver(model_list, f'models/integrable_model_L=5_bootstrap_seed{seed}.model')

[0.9964048049372081]
[0.9964108498579743]
[0.9964234940572]
[0.9964134187078276]
[0.9964061712955294]
[0.996401013212405]
[0.9963962859441235]
[0.9963785449190439]
[0.9964376169622816]
[0.9963795746406566]


In [9]:
path = '../data/chaos_exp_reorder/5q/integrable_L=5/'
n = 4
d = 2**n
L = 5
bootstrap_samples = 10

spam_list = []
model_list = []

i = 1 # second map

seed = 42 + i

inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

tf.random.set_seed(seed)
for j in range(bootstrap_samples):
    targets_spam_bs = resample(targets_spam, 12000)
    targets_map_bs = resample(targets_map, 12000)

    spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
    spam_list.append(spam_model) 


    model = fit_model(inputs_map, 
                    targets_map_bs, 
                    spam_model,
                    num_iter = 3000, 
                    verbose=False)
    model_list.append(model)


model_saver(spam_list, f'models/integrable_spam_L=5_bootstrap_seed{seed}.model')
model_saver(model_list, f'models/integrable_model_L=5_bootstrap_seed{seed}.model')

[0.9972516040525699]
[0.9972147529591883]
[0.9972047528882709]
[0.9972197545881452]
[0.9972568176942309]
[0.9972111569707001]
[0.9972200688042843]
[0.9971986087748004]
[0.9972137033400573]
[0.9972064745647305]


In [10]:
path = '../data/chaos_exp_reorder/5q/integrable_L=5/'
n = 4
d = 2**n
L = 5
bootstrap_samples = 10

spam_list = []
model_list = []

i = 2 # third map

seed = 42 + i

inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

tf.random.set_seed(seed)
for j in range(bootstrap_samples):
    targets_spam_bs = resample(targets_spam, 12000)
    targets_map_bs = resample(targets_map, 12000)

    spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
    spam_list.append(spam_model) 


    model = fit_model(inputs_map, 
                    targets_map_bs, 
                    spam_model,
                    num_iter = 3000, 
                    verbose=False)
    model_list.append(model)


model_saver(spam_list, f'models/integrable_spam_L=5_bootstrap_seed{seed}.model')
model_saver(model_list, f'models/integrable_model_L=5_bootstrap_seed{seed}.model')

[0.9968356277697868]
[0.996842936444056]
[0.9969098923181996]
[0.996876015530663]
[0.9967933446764149]
[0.9968510269853764]
[0.9968250056662179]
[0.9968616333528376]
[0.9968323390741513]
[0.9968487449995709]


In [11]:
path = '../data/chaos_exp_reorder/5q/integrable_L=5/'
n = 4
d = 2**n
L = 5
bootstrap_samples = 10

spam_list = []
model_list = []

i = 3 # fourth map

seed = 42 + i

inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

tf.random.set_seed(seed)
for j in range(bootstrap_samples):
    targets_spam_bs = resample(targets_spam, 12000)
    targets_map_bs = resample(targets_map, 12000)

    spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
    spam_list.append(spam_model) 


    model = fit_model(inputs_map, 
                    targets_map_bs, 
                    spam_model,
                    num_iter = 3000, 
                    verbose=False)
    model_list.append(model)


model_saver(spam_list, f'models/integrable_spam_L=5_bootstrap_seed{seed}.model')
model_saver(model_list, f'models/integrable_model_L=5_bootstrap_seed{seed}.model')

[0.9974760630964484]
[0.997454207374661]
[0.9974918288347829]
[0.9974922472121821]
[0.9974841576100824]
[0.997501523063753]
[0.9974737799000807]
[0.9974943815196148]
[0.997475059117183]
[0.9974626089242917]


In [ ]:
path = '../data/chaos_exp_reorder/5q/integrable_L=5/'
n = 4
d = 2**n
L = 5
bootstrap_samples = 10

for i in range(4, 10):

    spam_list = []
    model_list = []

    seed = 42 + i

    inputs_spam, targets_spam, inputs_map, targets_map, unitary = load_data(path + f'seed_{seed}.pkl', n, seed, L)

    tf.random.set_seed(seed)
    for j in range(bootstrap_samples):
        targets_spam_bs = resample(targets_spam, 12000)
        targets_map_bs = resample(targets_map, 12000)

        spam_model = fit_spam(inputs_spam, targets_spam_bs, verbose=False)
        spam_list.append(spam_model) 


        model = fit_model(inputs_map, 
                        targets_map_bs, 
                        spam_model,
                        num_iter = 3000, 
                        verbose=False)
        model_list.append(model)


    model_saver(spam_list, f'models/integrable_spam_L=5_bootstrap_seed{seed}.model')
    model_saver(model_list, f'models/integrable_model_L=5_bootstrap_seed{seed}.model')

[0.9964277043721148]
[0.996337451474645]
[0.9963912947137028]
[0.9963926820791307]
[0.9963378894973218]
[0.996408243742705]
[0.9963738454541801]
[0.9963363967344548]
[0.9964179812460542]
[0.9963592959670262]
[0.9963776353715253]
[0.9963730981540918]
[0.9964397766046494]
[0.9963844246188931]
[0.9964175496180987]
[0.9964200616236266]
[0.9964282022249626]
[0.9964384383602273]
[0.9964319275415643]
[0.996391146815959]
[0.9969550773536223]
[0.9969844847299797]
[0.996977976911108]
[0.9969925709184169]
[0.9969703509701064]
[0.9969879988620118]
[0.9969629349023501]
[0.9969851194749002]
[0.99699232021924]
[0.9969838110734919]
[0.9967398213012095]
[0.9966798487925761]
[0.9967305113282002]
[0.9967388749096527]
[0.9967337850998793]
[0.9966703542360308]
[0.9966939130003161]
[0.9967274389771538]
[0.9966686776805636]
[0.9967067496262231]
[0.9967955773061884]
[0.9967461742624427]
[0.9967665198345018]
[0.9967599080501204]
[0.9967819624232008]
[0.996760372652764]
[0.9967811849763506]
[0.9967974795048835]

: 